# Testing out my new models 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    trust_remote_code=True
)


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend